## BigQuery Vector Search with Clay v1.5 embeddings for NAIP
- ### Author: Biplov Bhandari

- ### Last Modified: December 6, 2025

- ### Description:
  This notebook demonstrates how to ingest geospatial embeddings hosted on AWS S3 into the Google Cloud ecosystem. It covers syncing data to Google Cloud Storage (GCS), loading Parquet files into BigQuery, and building a Vector Index (IVF) for semantic search.

## ⚠️ Cost Warning
Running this notebook may incur costs for the usuage of the following (Google) Cloud services.

- **Google Cloud Storage (GCS)**
- **BigQuery Storage**
- **BigQuery Compute**
- **AWS S3 (no cost)**

**Recommendation**: Delete the GCS bucket and BigQuery dataset after finishing the demo to avoid ongoing storage charges.

## Implementation

In [ ]:
# @title 1. Setup & Authentication
from google.colab import auth
auth.authenticate_user()

# Define your variables here
# "your-cloud-project-id"
project_id = "biplov-gde-project" # @param {type:"string"}
bucket_name = "clay-embeddings-v1-test" # @param {type:"string"}
region = "US" # @param {type:"string"}

# Data parameters
state = "wa"
year = 2021
mount_dir = f"mounted_{bucket_name}"

# BigQuery Table IDs
staging_table_id = f"{project_id}.naip_embeddings.raw_embeddings"
prod_table_id = f"{project_id}.naip_embeddings.embeddings"

print(f"Authenticated. Project: {project_id}, Bucket: {bucket_name}")

Authenticated. Project: biplov-gde-project, Bucket: clay-embeddings-v1-test


In [ ]:
# @title 2. Install Dependencies (AWS CLI & GCS Fuse)
%%bash
# Install AWS CLI
pip install awscli -q

# Install GCS Fuse (if not present)
export GCSFUSE_REPO=gcsfuse-`lsb_release -c -s`
echo "deb https://packages.cloud.google.com/apt $GCSFUSE_REPO main" | sudo tee /etc/apt/sources.list.d/gcsfuse.list
curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -
sudo apt-get update -q
sudo apt-get install gcsfuse -q

In [ ]:
# @title 3. Create Bucket and Mount GCS
import os

# Create the bucket if it doesn't exist
!gcloud storage buckets create gs://$bucket_name --project=$project_id --location=$region --uniform-bucket-level-access || echo "Bucket might already exist, skipping creation."

# Create the local mount directory
if not os.path.exists(mount_dir):
    os.makedirs(mount_dir)

# Mount the bucket using gcsfuse
!gcsfuse $bucket_name $mount_dir

In [ ]:
# @title 4. Download Data from Source Cooperative
# We use --no-sign-request because the S3 bucket is public
!aws s3 sync --no-sign-request s3://us-west-2.opendata.source.coop/clay/clay-v1-5-naip-2/$state/$year ./$mount_dir/naip_$state\_$year

In [ ]:
# @title 5. Create BigQuery Dataset
from google.cloud import bigquery

client = bigquery.Client(project=project_id)
# dataset_id = f"{project_id}.naip_embeddings"
dataset_id = ".".join(staging_table_id.split(".")[:-1])

dataset = bigquery.Dataset(dataset_id)
dataset.location = region

try:
    client.create_dataset(dataset, timeout=30)
    print(f"Created dataset {dataset_id}")
except Exception as e:
    print(f"Dataset {dataset_id} already exists or error: {e}")

Created dataset biplov-gde-project.naip_embeddings


In [ ]:
# @title 6. Load Data to Staging Table
def load_to_staging(gcs_uri: str):
    job_config = bigquery.LoadJobConfig(
        source_format=bigquery.SourceFormat.PARQUET,
        autodetect=True,
        write_disposition="WRITE_TRUNCATE",
    )
    load_job = client.load_table_from_uri(
        gcs_uri,
        staging_table_id,
        job_config=job_config
    )
    load_job.result()
    print(f"Staging table {staging_table_id} loaded from {gcs_uri}")

# Define the pattern to catch all files recursively
gcs_embeddings = f"gs://{bucket_name}/naip_{state}_{year}/*"

# Run the load
load_to_staging(gcs_embeddings)

Staging table biplov-gde-project.naip_embeddings.raw_embeddings loaded from gs://clay-embeddings-v1-test/naip_wa_2021/*


In [ ]:
# @title 7. Build or Update Production Table
def build_or_append_prod(state: str, year: int):
    """
    Appends data to prod table. Creates table if it doesn't exist.
    Refreshes data for the specific state/year to prevent duplicates.
    """

    # 1. Create the table if it doesn't exist (Idempotent)
    create_sql = f"""
    CREATE TABLE IF NOT EXISTS `{prod_table_id}`
    (
        geometry GEOGRAPHY,
        embeddings ARRAY<FLOAT64>,
        state STRING,
        year INT64,
        dt DATE,
        geohash5 STRING
    )
    PARTITION BY dt
    CLUSTER BY geohash5
    """
    client.query(create_sql).result()

    # 2. Delete existing data for this specific state/year (to allow re-runs)
    delete_sql = f"""
    DELETE FROM `{prod_table_id}`
    WHERE state = '{state}' AND year = {year}
    """
    client.query(delete_sql).result()

    # 3. Insert the new data
    insert_sql = f"""
    INSERT INTO `{prod_table_id}`
    SELECT
      geometry,
      -- flatten the nested RECORD into a true ARRAY<FLOAT64>
      ARRAY(SELECT r.element FROM UNNEST(embeddings.list) AS r) AS embeddings,
      "{state}" AS state,
      {year}     AS year,
      DATE({year}, 1, 1) AS dt,
      ST_GEOHASH(ST_CENTROID(geometry), 5) AS geohash5
    FROM `{staging_table_id}`;
    """
    client.query(insert_sql).result()

    print(f"Production table {prod_table_id} UPDATED for {state}/{year}.")

build_or_append_prod(state=state, year=year)

Production table biplov-gde-project.naip_embeddings.embeddings UPDATED for wa/2021.


In [ ]:
# @title 8. Create Vector Index
index_sql = f"""
CREATE VECTOR INDEX IF NOT EXISTS naip_index ON `{prod_table_id}`(embeddings)
OPTIONS(distance_type='COSINE', index_type='IVF', ivf_options='{{"num_lists": 1000}}');
"""

client.query(index_sql).result()
print("Vector index creation initiated. This may take a few minutes.")

Vector index creation initiated. This may take a few minutes.


In [ ]:
# @title 9. Check Index Status
from google.cloud.bigquery import magics
# Set the context for the magic command
magics.context.project = project_id

In [ ]:
dataset_name = prod_table_id.split(".")[1]
dataset_name

'naip_embeddings'

In [ ]:
%%bigquery
# dataset_name provided above (in this case, naip_embeddings)
SELECT * FROM naip_embeddings.INFORMATION_SCHEMA.VECTOR_INDEXES;


Query is running:   0%|          |

Downloading:   0%|          |

,index_catalog,index_schema,table_name,index_name,index_status,creation_time,last_modification_time,last_refresh_time,disable_time,disable_reason,ddl,coverage_percentage,unindexed_row_count,total_logical_bytes,total_storage_bytes
0,biplov-gde-project,naip_embeddings,embeddings,naip_index,ACTIVE,2025-12-06 13:31:02.424000+00:00,2025-12-06 13:31:02.424000+00:00,2025-12-06 13:38:07.011000+00:00,NaT,None,CREATE VECTOR INDEX `naip_index` ON `biplov-gd...,100,0,42523200886,23215362610


In [ ]:
# @title 10. Demo: Run Vector Search (Find Similar Landscapes)
# In a real app, you would generate an embedding for a user's query.
# You can use Earth Engine UI to do this
# example script: https://github.com/biplovbhandari/g4g-25-demos/blob/main/src/ee-ui.js

# For this demo, we will pick one random tile from the table and find its
# nearest neighbors to simulate finding "similar terrain".

search_sql = f"""
SELECT
  query.geohash5 AS query_geohash,
  base.geohash5 AS similar_geohash,
  base.state,
  base.dt,
  distance
FROM
  VECTOR_SEARCH(
    TABLE `{prod_table_id}`,
    'embeddings',
    (SELECT * FROM `{prod_table_id}` LIMIT 1),
    top_k => 10,
    distance_type => 'COSINE'
  )
"""

# Run the search
results = client.query(search_sql).to_dataframe()

print("Top 10 Similar Tiles Found:")
print(results.to_markdown(index=False))

Top 10 Similar Tiles Found:
| query_geohash   | similar_geohash   | state   | dt         |   distance |
|:----------------|:------------------|:--------|:-----------|-----------:|
| c23fp           | c23fp             | wa      | 2021-01-01 |  0         |
| c23fp           | c21yq             | wa      | 2021-01-01 |  0.0242282 |
| c23fp           | c23fp             | wa      | 2021-01-01 |  0.0257341 |
| c23fp           | c26hy             | wa      | 2021-01-01 |  0.0262999 |
| c23fp           | c23fp             | wa      | 2021-01-01 |  0.0270856 |
| c23fp           | c2642             | wa      | 2021-01-01 |  0.0274472 |
| c23fp           | c23fp             | wa      | 2021-01-01 |  0.0292352 |
| c23fp           | c2edu             | wa      | 2021-01-01 |  0.0298167 |
| c23fp           | c23fx             | wa      | 2021-01-01 |  0.0298949 |
| c23fp           | c23fr             | wa      | 2021-01-01 |  0.0304455 |
